In [1]:
import numpy as np
import random
import pandas as pd
import scipy.io
import matplotlib.pyplot as plt
from qiskit.quantum_info import DensityMatrix, random_density_matrix
from qiskit.quantum_info.operators import Operator
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from scipy.linalg import sqrtm

# 1) 2 Qubit Analysis

## 1.1) Load The Datasets

In [ ]:
freq_2q = np.load('../data/2_qubit_ma/noise/num100000_cube_9_N=100000_noise0.0100.npy')
print(f'2 Qubit Frequency Data Shape: {freq_2q.shape}')

rho_2 = np.load('../data/2_qubit_ma/level4num100000_pure_haar.npy')
print(rho_2.shape)
print(rho_2)

(100000, 36)
(100000, 4, 4)
[[[ 0.21499355+0.j         -0.30373088-0.0047481j
   -0.14253069-0.02100995j -0.14300953-0.18785141j]
  [-0.30373088+0.0047481j   0.42919896+0.j
    0.20182343+0.02653391j  0.20618457+0.26222767j]
  [-0.14253069+0.02100995j  0.20182343-0.02653391j
    0.0965444 +0.j          0.11316618+0.11056132j]
  [-0.14300953+0.18785141j  0.20618457-0.26222767j
    0.11316618-0.11056132j  0.25926308+0.j        ]]

 [[ 0.20897381+0.j         -0.3062431 -0.14170762j
    0.06937363-0.162879j   -0.03196807-0.13810725j]
  [-0.3062431 +0.14170762j  0.54488121+0.j
    0.00878579+0.28573603j  0.14050016+0.18071297j]
  [ 0.06937363+0.162879j    0.00878579-0.28573603j
    0.14998184+0.j          0.09703145-0.07076451j]
  [-0.03196807+0.13810725j  0.14050016-0.18071297j
    0.09703145+0.07076451j  0.09616314+0.j        ]]

 [[ 0.21460249+0.j         -0.187629  -0.2088956j
   -0.05586883-0.01352221j  0.14962502+0.25301065j]
  [-0.187629  +0.2088956j   0.36738636+0.j
    0.06200927-0